<a href="https://colab.research.google.com/github/auroramugnai/ArXivClassification/blob/main/ArXivClassification/keywords_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the github repository and move to the inner directory.

In [ ]:
!git clone https://github.com/auroramugnai/ArXivClassification.git
%cd ArXivClassification/ArXivClassification

Cloning into 'ArXivClassification'...
remote: Enumerating objects: 1718, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 1718 (delta 285), reused 271 (delta 230), pack-reused 1358
Receiving objects: 100% (1718/1718), 23.17 MiB | 18.06 MiB/s, done.
Resolving deltas: 100% (900/900), done.
/content/ArXivClassification/ArXivClassification


# 1) Build the dataset

In [ ]:
import json
import random
import zipfile

import dask.bag as db
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils

In [ ]:
SEED = 42 # fix random seed for reproducibility
random.seed(SEED)

## 1.1 Download the dataset
The following line of code comes from clicking on "Copy API command" in https://www.kaggle.com/datasets/Cornell-University/arxiv.

In [ ]:
!kaggle datasets download -d Cornell-University/arxiv

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
100% 1.28G/1.28G [00:35<00:00, 40.5MB/s]
100% 1.28G/1.28G [00:35<00:00, 38.6MB/s]


Unzip the downloaded file.

In [ ]:
with zipfile.ZipFile('./arxiv.zip', 'r') as zip_ref:
    zip_ref.extractall()

The unzipping creates an "arxiv-metadata-oai-snapshot.json". We now create a dask bag out of it.

In [ ]:
path = "./arxiv-metadata-oai-snapshot.json"
arxiv_data = db.read_text(path).map(json.loads)
arxiv_data.take(1)

({'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

## 1.2 Get rid of some unnecessary information

In [ ]:
# Get the latest version of the articles.
get_latest_version = lambda x: x['versions'][-1]['created']

# Only keep articles published after 2022.
is_after_2020 = lambda x: int(get_latest_version(x).split(' ')[3]) > 2020

# Only keep some information.
cut_info = lambda x: {'id': x['id'],
                      'title': x['title'],
                      'category':x['categories'].split(' '),
                      'abstract':x['abstract'],}

# Only keep Computer Science macro-category.
is_only_cs = lambda x: all([s.startswith("cs.") for s in x['categories'].split(' ')])

arxiv_data_filtered = (arxiv_data.filter(is_after_2020).filter(is_only_cs).map(cut_info).compute())


# Create a pandas dataframe and save it to csv.
df = pd.DataFrame(arxiv_data_filtered)
df.to_csv("./cs_arxiv_data_filtered.csv", index=False)
df.head()

,id,title,category,abstract
0,0710.3901,A recursive linear time modular decomposition ...,[cs.DM],A module of a graph G is a set of vertices t...
1,0711.2010,A Polynomial Time Algorithm for Graph Isomorphism,[cs.CC],We claimed that there is a polynomial algori...
2,0802.3414,A Universal In-Place Reconfiguration Algorithm...,"[cs.CG, cs.MA, cs.RO]",In the modular robot reconfiguration problem...
3,0803.3946,On the `Semantics' of Differential Privacy: A ...,"[cs.CR, cs.DB]","Differential privacy is a definition of ""pri..."
4,0805.1877,Perfect tag identification protocol in RFID ne...,[cs.NI],Radio Frequency IDentification (RFID) system...


## 1.3 Get a fixed number of articles
To speed up computation and avoid a session crash.

In [ ]:
path = "./cs_arxiv_data_filtered.csv"
df = pd.read_csv(path, dtype=str)
df.head()

,id,title,category,abstract
0,0710.3901,A recursive linear time modular decomposition ...,['cs.DM'],A module of a graph G is a set of vertices t...
1,0711.2010,A Polynomial Time Algorithm for Graph Isomorphism,['cs.CC'],We claimed that there is a polynomial algori...
2,0802.3414,A Universal In-Place Reconfiguration Algorithm...,"['cs.CG', 'cs.MA', 'cs.RO']",In the modular robot reconfiguration problem...
3,0803.3946,On the `Semantics' of Differential Privacy: A ...,"['cs.CR', 'cs.DB']","Differential privacy is a definition of ""pri..."
4,0805.1877,Perfect tag identification protocol in RFID ne...,['cs.NI'],Radio Frequency IDentification (RFID) system...


In [ ]:
num_data = 10000 # number of articles that we want to keep
print(f"The dataset contains {len(df)} articles.")

# Sample the dataset only if its length exceeds num_data.
if(len(df) > num_data):
    n_sample = num_data
    df = df.sample(n=n_sample, axis=0, random_state=SEED)

print(f"The dataset contains {len(df)} articles.")

The dataset contains 201390 articles.
The dataset contains 10000 articles.


# 2) Text-processing

In [ ]:
!pip install -U spacy -q
!python -m spacy download en_core_web_md -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import en_core_web_md
import spacy
from tqdm import tqdm

Clean out the strings (this step will take a while).

In [ ]:
# Remove stop words, punctuation, special characters, numbers.
nlp = spacy.load("en_core_web_md")
tqdm.pandas() # to display progress bar

# First on abstracts.
abs_cleaner = lambda x: utils.text_cleaner(text=x["abstract"], nlp=nlp)
df["clean_abstract"] = df.progress_apply(abs_cleaner, axis=1)

# Then on titles.
tit_cleaner = lambda x: utils.text_cleaner(text=x["title"], nlp=nlp)
df["clean_title"] = df.progress_apply(tit_cleaner, axis=1)

df.tail()

100%|██████████| 10000/10000 [01:17<00:00, 128.29it/s]


,id,title,category,abstract,clean_abstract,clean_title
120147,2303.14329,Edge-Based Video Analytics: A Survey,"['cs.DC', 'cs.CV']",Edge computing has been getting a momentum w...,edge computing momentum increase datum edge ne...,edge base video analytics survey
139871,2307.04054,Deep Unsupervised Learning Using Spike-Timing-...,['cs.CV'],Spike-Timing-Dependent Plasticity (STDP) is ...,spike timing dependent plasticity stdp unsuper...,deep unsupervised learning use spike timing de...
138750,2307.00182,Single-Stage Heavy-Tailed Food Classification,['cs.CV'],Deep learning based food image classificatio...,deep learning base food image classification e...,single stage heavy tailed food classification
33080,2106.16020,Anomaly Detection: How to Artificially Increas...,['cs.LG'],Anomaly detection is a widely explored domai...,anomaly detection widely explore domain machin...,anomaly detection artificially increase score ...
31726,2106.10938,A Game-Theoretic Taxonomy of Visual Concepts i...,"['cs.LG', 'cs.CV']","In this paper, we rethink how a DNN encodes ...",paper rethink dnn encode visual concept differ...,game theoretic taxonomy visual concepts dnns


In [ ]:
# Add a space to separate title and abstract.
df["clean_text"] = df["clean_title"] + " " + df["clean_abstract"]
df.head()

,id,title,category,abstract,clean_abstract,clean_title,clean_text
82985,2207.05068,Few-Shot Semantic Relation Prediction across H...,"['cs.LG', 'cs.AI']",Semantic relation prediction aims to mine th...,semantic relation prediction aim implicit rela...,shot semantic relation prediction heterogeneou...,shot semantic relation prediction heterogeneou...
29411,2106.02954,Denoising Word Embeddings by Averaging in a Sh...,"['cs.CL', 'cs.LG']",We introduce a new approach for smoothing an...,introduce new approach smooth improve quality ...,denoise word embeddings average shared space,denoise word embeddings average shared space i...
128648,2305.08492,On the conformance of Android applications wit...,"['cs.CY', 'cs.CR']",With the rapid development of online technol...,rapid development online technology widespread...,conformance android application child data pro...,conformance android application child data pro...
87965,2208.08934,A Hybrid Self-Supervised Learning Framework fo...,['cs.LG'],"Vertical federated learning (VFL), a variant...",vertical federated learning vfl variant federa...,hybrid self supervised learning framework vert...,hybrid self supervised learning framework vert...
75162,2205.08982,Attention-based Multimodal Feature Representat...,['cs.IR'],"In recommender systems, models mostly use a ...",recommender system model use combination embed...,attention base multimodal feature representati...,attention base multimodal feature representati...


# 3) Keywords extraction

In [ ]:
!pip install KeyBERT -q
!pip install keyphrase-vectorizers -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.5/363.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.8/772.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 65.3 MB/s eta 0:00:00


In [ ]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
kw_model = KeyBERT('all-mpnet-base-v2')

extraction = lambda x: utils.extract_kws(text=x["clean_text"],
                                         kw_model=kw_model,
                                         seed=x["clean_title"].split(" "))

df["keywords"] = df.progress_apply(extraction, axis=1)
df.head()

100%|██████████| 10000/10000 [17:11<00:00,  9.69it/s]


,id,title,category,abstract,clean_abstract,clean_title,clean_text,keywords
82985,2207.05068,Few-Shot Semantic Relation Prediction across H...,"['cs.LG', 'cs.AI']",Semantic relation prediction aims to mine th...,semantic relation prediction aim implicit rela...,shot semantic relation prediction heterogeneou...,shot semantic relation prediction heterogeneou...,"[subgraph, semantic, metags, predict]"
29411,2106.02954,Denoising Word Embeddings by Averaging in a Sh...,"['cs.CL', 'cs.LG']",We introduce a new approach for smoothing an...,introduce new approach smooth improve quality ...,denoise word embeddings average shared space,denoise word embeddings average shared space i...,"[embeddings, word, denoise, average]"
128648,2305.08492,On the conformance of Android applications wit...,"['cs.CY', 'cs.CR']",With the rapid development of online technol...,rapid development online technology widespread...,conformance android application child data pro...,conformance android application child data pro...,"[gdpr, guideline, android, child]"
87965,2208.08934,A Hybrid Self-Supervised Learning Framework fo...,['cs.LG'],"Vertical federated learning (VFL), a variant...",vertical federated learning vfl variant federa...,hybrid self supervised learning framework vert...,hybrid self supervised learning framework vert...,"[federated, learning, privacy, vertical]"
75162,2205.08982,Attention-based Multimodal Feature Representat...,['cs.IR'],"In recommender systems, models mostly use a ...",recommender system model use combination embed...,attention base multimodal feature representati...,attention base multimodal feature representati...,"[multimodal, recommender, attention, embed]"


# 4) Classification
Given an article:

- its feature X will be the cleaned text
- its label y will be its keyword

In [ ]:
!pip install scikit-multilearn -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [ ]:
# Preparing X (features)
X = df["clean_text"]

# Preparing y (labels)
y = df['keywords']

#Split data into train/test.
X_train, X_test, y_train_tot, y_test_tot = train_test_split(X, y,
                                                            test_size=0.5,
                                                            random_state=SEED)

In [ ]:
# Select only the first keyword for every article.
y_train = [x[0] for x in y_train_tot]
y_test = [x[0] for x in y_test_tot]

Do the classification.

In [ ]:
model = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                  ('svm_model', LinearSVC(verbose=1))])

y_pred = utils.run_model(model, X_train, X_test, y_train, y_test,
                         multilabel=False)

print('accuracy: ', accuracy_score(y_test, y_pred))

df_pred = pd.DataFrame({'clean_text': X_test,
                        'true_kws': y_test_tot,
                        'first_true_kw': y_test,
                        'predicted_kw': y_pred})

[LibLinear]accuracy:  0.349


In [ ]:
df_pred = df_pred.reset_index(drop=True)
df_pred.head()

,clean_text,true_kws,first_true_kw,predicted_kw
0,locally fair partitioning model societal task ...,"[partitioning, district, fairness, cooperative]",partitioning,federate
1,multiple subset problem encryption scheme comm...,"[encryption, subset, problem, mssp]",encryption,encryption
2,co nnect framework reveal commonsense knowledg...,"[conceptnet, classifier, commonsense, text]",conceptnet,commonsense
3,fast bitmap fit cpu cache line friendly memory...,"[allocator, bitmap, memory, randomization]",allocator,memory
4,neuda neural deformable anchor high fidelity i...,"[neural, surface, voxel, casting]",neural,nodes


In [ ]:
# Get the number of predicted kws that are contained in the list of true kws.
is_in_true_kws = lambda x: x.predicted_kw in x.true_kws
num_true = df_pred.apply(is_in_true_kws, axis=1).value_counts().loc[True]

# Turn it to percentage.
print(f"{round((num_true/len(df_pred))*100, 2)}% of predicted kws are true kws")

53.04% of predicted kws are true kws


# 5) Compute the distance between the true and the predicted keywords

In [ ]:
import nltk
from gensim.models import Word2Vec

In [ ]:
# Lists of the keywords on which we want to compute the similarity.
kws_pred = df_pred['predicted_kw'].values
kws_true = df_pred['first_true_kw'].values

In [ ]:
# Create the corpus using our processed texts.
corpus = list(df['clean_text'].values)

# Tokenize the corpus.
nltk.download('punkt')
tokenized_corpus = [nltk.word_tokenize(text.lower()) for text in corpus]

# Train the Word2Vec model on the created corpus.
model = Word2Vec(tokenized_corpus, min_count=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Compute the meaning similarity.

In [ ]:
simil_meaning_list = [] # meaninig similarity

for i, (kp, kb) in enumerate(zip(kws_pred, kws_true)):
    sim = model.wv.similarity(kp, kb)
    sim = float("{0:.2f}".format(sim))
    if (i<5): print(f"The similarity between '{kp}' and '{kb}' is: {sim}")
    simil_meaning_list.append(sim)

print(f"\nMEAN OF SIMILARITIES: {np.mean(simil_meaning_list)}")

The similarity between 'federate' and 'partitioning' is: 0.37
The similarity between 'encryption' and 'encryption' is: 1.0
The similarity between 'commonsense' and 'conceptnet' is: 0.65
The similarity between 'memory' and 'allocator' is: 0.74
The similarity between 'nodes' and 'neural' is: 0.25

MEAN OF SIMILARITIES: 0.637372
